In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import scipy
import sys

from rdkit import Chem
from rdkit.Chem.Lipinski import NumAromaticRings
from rdkit.Chem import PandasTools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# 读入各个数据
latent = pd.read_table('./data/2105_confirm/latent_features.txt', sep='.\s', header=None, engine='python')
smiles = pd.read_table('./data/2105_confirm/z5.txt',sep='\n', header=None)
data_all = pd.concat([smiles, latent], axis=1)
col = ['smiles']
name = 'jtvae_ld_'
for i in range(56):
    col.append(name + str(i))
data_all.columns = col


In [3]:
data_all

,smiles,jtvae_ld_0,jtvae_ld_1,jtvae_ld_2,jtvae_ld_3,jtvae_ld_4,jtvae_ld_5,jtvae_ld_6,jtvae_ld_7,jtvae_ld_8,...,jtvae_ld_46,jtvae_ld_47,jtvae_ld_48,jtvae_ld_49,jtvae_ld_50,jtvae_ld_51,jtvae_ld_52,jtvae_ld_53,jtvae_ld_54,jtvae_ld_55
0,Cc1c(C(=O)N[C@H](C[NH+](C)C)c2ccccc2)cnn1-c1cc...,-1.261103,1.888711,-6.242961,-6.217049,-1.963670,2.881199,-1.815423,2.818454,-5.236242,...,-3.111375,1.120334,-2.613182,2.387751,5.268334,2.352951,-3.541683,-2.524042,7.874137,0.189427
1,COc1ccc(C[NH+](C)[C@@H](C)C(=O)Nc2ccc(F)c(F)c2...,2.526336,-2.564474,-3.490833,4.425713,-2.487651,1.911674,1.005830,-2.580270,-3.847111,...,-4.105083,2.813607,4.779819,1.010575,6.866753,-1.441126,-4.302085,-5.931015,-1.912545,-0.369789
2,CC[NH+](CCS(C)(=O)=O)Cc1ccc(-c2ccc(Cl)cc2)o1,1.788244,-9.902813,-2.343237,2.489923,2.706184,3.073043,2.884255,-2.205066,7.632130,...,3.991690,2.795439,3.964744,5.913845,6.573098,-2.178797,7.590621,-8.152710,7.147285,-0.221404
3,CN1C(=O)/C(=C(\Sc2nnc3c(n2)[nH]c2ccccc23)C(=O)...,-1.428424,2.735042,-4.686931,4.330184,2.785543,-6.069830,6.248152,1.152485,-1.735201,...,1.654738,-2.257859,-3.779827,5.830238,3.428591,2.706142,3.741402,3.237207,1.102386,-0.089020
4,COc1cccc(C[NH2+]C[C@H]2CCN(CC(F)(F)F)C2)c1,4.793871,-2.767154,-2.875158,2.710938,-1.471237,9.194351,-1.084160,-1.066807,2.845645,...,-5.026968,5.103010,8.291696,4.514052,2.214745,-3.881510,6.646420,-5.560030,-1.020045,0.095746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,CSc1ccccc1NC(=O)[C@H]1CC[C@@H]2CCCC[C@@H]2[NH2+]1,-1.071066,1.715748,1.629091,3.196286,-2.792036,-5.045714,-9.558347,-3.738821,-1.076215,...,1.651217,2.902797,-4.159595,9.998109,-1.511694,2.492025,4.853007,2.112207,3.294874,0.295081
49996,Cc1cc([C@]2(O)CS[C@H](C)C2)ccc1F,5.500004,-4.974154,-1.010477,1.012926,-2.121687,-2.464846,8.930106,-4.507516,-6.026894,...,3.552136,4.714711,-3.645862,-3.396438,6.527954,7.050245,9.533513,-7.430381,1.574888,0.413131
49997,CN(CC1CC1)C(=O)NCC(C)(C)c1ccc(Cl)cc1,3.877814,-1.258259,-2.414311,-9.483868,-9.757860,4.956900,1.118094,1.629370,-9.229904,...,-2.426497,-3.229744,1.192198,5.310479,6.923873,-4.689966,1.704111,-4.871596,-3.132816,0.115074
49998,O=C([O-])CCc1ccc(S(=O)(=O)Nc2cccc(I)c2)cc1,1.091118,-4.024869,-1.806850,5.785818,-3.052545,4.511624,-1.724618,-1.059059,4.428439,...,-2.388021,-1.569831,-1.008806,2.462220,2.819120,-4.666101,5.199871,-2.651287,-4.078031,-0.273533


In [4]:
def get_aroma(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return NumAromaticRings(mol)

data_all.set_index(['smiles'], inplace=True)
data_index = pd.DataFrame(smiles)
data_index.columns = ['smiles']
data_index['aroma'] = data_index['smiles'].apply(lambda x: get_aroma(x))
data_index.set_index(['smiles'], inplace=True)

In [5]:
data_index

,aroma
smiles,
Cc1c(C(=O)N[C@H](C[NH+](C)C)c2ccccc2)cnn1-c1ccccc1,3
COc1ccc(C[NH+](C)[C@@H](C)C(=O)Nc2ccc(F)c(F)c2F)cc1,2
CC[NH+](CCS(C)(=O)=O)Cc1ccc(-c2ccc(Cl)cc2)o1,2
CN1C(=O)/C(=C(\Sc2nnc3c(n2)[nH]c2ccccc23)C(=O)[O-])c2ccccc21,4
COc1cccc(C[NH2+]C[C@H]2CCN(CC(F)(F)F)C2)c1,1
...,...
CSc1ccccc1NC(=O)[C@H]1CC[C@@H]2CCCC[C@@H]2[NH2+]1,1
Cc1cc([C@]2(O)CS[C@H](C)C2)ccc1F,1
CN(CC1CC1)C(=O)NCC(C)(C)c1ccc(Cl)cc1,1


In [6]:
# set A = 2 rings, set B = 1 or 3 rings
mols_to_stay = np.logical_or.reduce((data_index['aroma'].values == 1, data_index['aroma'].values == 2, data_index['aroma'].values == 3))
df_experiment = data_index.iloc[mols_to_stay]
print("Number of molecules with 1/2/3 aromatic rings: %d" % df_experiment.shape[0])

Number of molecules with 1/2/3 aromatic rings: 44411


In [7]:
# Split molecules into dataset A and B
A_indices = df_experiment['aroma'].values == 2

df_A = df_experiment.iloc[A_indices]
df_B = df_experiment.iloc[~A_indices]

print("Number of molecules in dataset A: %d" % df_A.shape[0])
print("Number of molecules in dataset B: %d" % df_B.shape[0])

Number of molecules in dataset A: 19762
Number of molecules in dataset B: 24649


In [9]:
# Sanity check of datasets
assert set(df_A['aroma'].values) == set([2])
assert set(df_B['aroma'].values) == set([1,3])

In [10]:
# Split datasets into train and test datasets.
train_size = 12000

# split set A to train/test
train_sample_A = np.zeros(df_A.shape[0], dtype=bool)
train_sample_A[np.random.choice(df_A.shape[0], train_size, replace=False)] = True

df_train_A = df_A.iloc[train_sample_A]
df_test_A = df_A.iloc[~train_sample_A]
print("Number of molecules in dataset train_A: %d" % df_train_A.shape[0])
print("Number of molecules in dataset test_A: %d" % df_test_A.shape[0])

# split set B to train/test
train_sample_B = np.zeros(df_B.shape[0], dtype=bool)
train_sample_B[np.random.choice(df_B.shape[0], train_size, replace=False)] = True

df_train_B = df_B.iloc[train_sample_B]
df_test_B = df_B.iloc[~train_sample_B]
print("Number of molecules in dataset train_B: %d" % df_train_B.shape[0])
print("Number of molecules in dataset test_B: %d" % df_test_B.shape[0])

Number of molecules in dataset train_A: 12000
Number of molecules in dataset test_A: 7762
Number of molecules in dataset train_B: 12000
Number of molecules in dataset test_B: 12649


In [11]:
X_JTVAE_zinc_train_A = data_all.loc[df_train_A.index]
X_JTVAE_zinc_test_A = data_all.loc[df_test_A.index]

X_JTVAE_zinc_train_B = data_all.loc[df_train_B.index]
X_JTVAE_zinc_test_B = data_all.loc[df_test_B.index]

print(X_JTVAE_zinc_train_A.shape, X_JTVAE_zinc_test_A.shape, 
      X_JTVAE_zinc_train_B.shape, X_JTVAE_zinc_test_B.shape)

(12000, 56) (7762, 56) (12000, 56) (12649, 56)


In [14]:
X_JTVAE_zinc_train_A.to_csv('./data/2105_confirm/X_JTVAE_zinc_train_A.csv')
X_JTVAE_zinc_test_A.to_csv('./data/2105_confirm/X_JTVAE_zinc_test_A.csv')
X_JTVAE_zinc_train_B.to_csv('./data/2105_confirm/X_JTVAE_zinc_train_B.csv')
X_JTVAE_zinc_test_B.to_csv('./data/2105_confirm/X_JTVAE_zinc_test_B.csv')